# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import scipy.stats as st
from scipy.stats import linregress
from pprint import pprint
from colorama import Fore
from colorama import Style
import datetime
import json
x = datetime.datetime.now()


# Automatically set Data Frames to 2 decimal places
pd.options.display.precision = 2

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

# print(x.strftime("%d""/""%m""/""%Y"))

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# File to Load 
vacation = "../output_data/temperature_file.csv"

# Read file and store into Pandas data frame
vacation_df = pd.read_csv(vacation)
vacation_df

,City,Country,Temperature (f),Latitude,Longitude,Humidity (%),Cloudiness (%),Wind Speed (mph)
0,Ust-Tsilma,RU,56.37,65.44,52.15,70,100,8.72
1,Hithadhoo,MV,84.85,-0.60,73.08,70,85,6.91
2,Busselton,AU,55.20,-33.65,115.33,82,75,17.52
3,Margate,GB,65.66,51.38,1.39,64,93,3.00
4,Rondonópolis,BR,89.49,-16.47,-54.64,29,11,7.70
...,...,...,...,...,...,...,...,...
557,Nelson Bay,AU,52.48,-32.72,152.15,71,40,9.17
558,Puerto Carreño,CO,77.00,6.19,-67.49,88,20,5.82
559,Chegdomyn,RU,64.78,51.12,133.02,90,100,0.83
560,Necochea,AR,44.01,-38.55,-58.74,80,4,5.01


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = vacation_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float for Humidity
humidity = vacation_df["Humidity (%)"].astype(float)

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating =False, max_intensity=10, point_radius=1)

# Add heat layer to figure
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# View statistics of the data frame to view what conditions we have available, then select my ideal weather conditions
vacation_df.describe()

,Temperature (f),Latitude,Longitude,Humidity (%),Cloudiness (%),Wind Speed (mph)
count,562.00,562.00,562.00,562.00,562.00,562.00
mean,69.15,20.10,21.13,69.07,48.45,7.80
std,15.79,33.59,92.39,22.25,39.51,4.81
min,26.60,-54.80,-179.17,3.00,0.00,0.36
25%,57.25,-8.08,-61.18,58.00,4.00,4.00
50%,69.97,21.32,24.96,75.00,40.00,6.93
75%,80.72,50.24,108.08,86.00,90.00,10.29
max,115.56,78.22,179.32,100.00,100.00,27.16


In [7]:
# Ideal weather conditions are:
# Temperature not less than 86(f) but no more than 104(f)
# Wind speed less than 6(mph)
# Cloudiness less than 5%

# Filter the data frame and create a new one showing only the parameters listed above
ideal_weather = vacation_df.loc[(vacation_df["Temperature (f)"]>86)&(vacation_df["Temperature (f)"]<104)
                                &(vacation_df["Wind Speed (mph)"]<6)
                                &(vacation_df["Cloudiness (%)"]<5)]

# View statistics of the new data frame to ensure we have a new data frame with only the ideal selected weather conditions captured
ideal_weather.describe()

,Temperature (f),Latitude,Longitude,Humidity (%),Cloudiness (%),Wind Speed (mph)
count,4.00,4.00,4.00,4.00,4.00,4.00
mean,87.56,11.78,-35.78,47.50,0.25,3.69
std,1.19,16.03,47.97,14.57,0.50,1.15
min,86.59,-2.04,-66.91,30.00,0.00,2.42
25%,86.96,-0.82,-65.49,38.25,0.00,2.85
50%,87.18,8.78,-55.49,50.00,0.00,3.81
75%,87.78,21.38,-25.78,59.25,0.25,4.65
max,89.29,31.61,34.76,60.00,1.00,4.70


In [8]:
# View new data frame
ideal_weather.reset_index()
ideal_weather

,City,Country,Temperature (f),Latitude,Longitude,Humidity (%),Cloudiness (%),Wind Speed (mph)
175,Guánica,PR,86.59,17.97,-66.91,41,0,4.63
267,Maracaçumé,BR,87.28,-2.04,-45.96,59,1,4.70
314,Santa Isabel Rio Negro,BR,87.08,-0.41,-65.02,60,0,2.42
492,Kiryat Gat,IL,89.29,31.61,34.76,30,0,3.00


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Store into variable named hotel_df
hotel_df = ideal_weather

# Add a "Hotel Name column to the data frame"
hotel_df["Hotel Name"] = ""

# Set parameters to search for hotels within 5000 meters of each city
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# create a params dict that will be updated with new city each iteration
params = {"key": g_key,
        "radius": 5000,
        "types": "lodging",
        "keyword": "hotel",
        "key": g_key}

# Loop through the cities to find the first hotel
for index, row in hotel_df.iterrows():
    
    # get city name from data frame
    city = row["City"]
    
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']

# Print results to check 
# results

    try:
        print(f"A hotel in {city} is: {results[0]['name']}.")
        
        # Add hotel name to data frame
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
         
        
    except (KeyError, IndexError):
        print("No hotel found for city... skipping.")
        
    print("------------")
    


Retrieving Results for: Guánica.
A hotel in Guánica is: Copamarina Beach Resort & Spa.
------------
Retrieving Results for: Maracaçumé.
A hotel in Maracaçumé is: HOTEL MODELO.
------------
Retrieving Results for: Santa Isabel Rio Negro.
A hotel in Santa Isabel Rio Negro is: Hotel Ayuana.
------------
Retrieving Results for: Kiryat Gat.
A hotel in Kiryat Gat is: Desert Gat.
------------


In [10]:
hotel_df.head()

,City,Country,Temperature (f),Latitude,Longitude,Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
175,Guánica,PR,86.59,17.97,-66.91,41,0,4.63,Copamarina Beach Resort & Spa
267,Maracaçumé,BR,87.28,-2.04,-45.96,59,1,4.70,HOTEL MODELO
314,Santa Isabel Rio Negro,BR,87.08,-0.41,-65.02,60,0,2.42,Hotel Ayuana
492,Kiryat Gat,IL,89.29,31.61,34.76,30,0,3.00,Desert Gat


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [12]:
# Create and add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))